# <center>OpenAI在线大模型调用及微调方法

## <center>Ch.13 用AI开发AI（中）：全自动函数编写策略

&emsp;&emsp;在上一小节中，我们尝试了在本地搭建Chat模型代码生成、验证和保存的全套流程，进而辅助完成高效的代码开发工作。接下来，我们进一步深化大模型在实际开发过程中的作用，即尝试梳理当前开发流程中的核心环节，并尝试借助Chat模型完成各核心环节的工作。

- 软件开发过程利用大语言模型提高开发效率的不同阶段

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/702fe4b67f1781de14a5e217a2d923b.png" alt="702fe4b67f1781de14a5e217a2d923b" style="zoom:33%;" />

In [1]:
import os
import openai
import glob
import shutil
openai.api_key = os.getenv("OPENAI_API_KEY")

import numpy as np
import pandas as pd

import json
import io
import inspect
import requests
import re

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import base64
import email
from email import policy
from email.parser import BytesParser
from email.mime.text import MIMEText

from bs4 import BeautifulSoup
import dateutil.parser as parser

import sys
sys.path.insert(0, '.\\functions\\untested functions')
sys.path.insert(0, '.\\functions\\tested functions')
from gptLearning import *

In [2]:
def retrieve_emails(n, user_id='me'):
    """
    获取指定数量的最近邮件。

    参数:
    n: 要检索的邮件的数量。这应该是一个整数。
    user_id: 要检索邮件的用户的ID。默认值是'me'，表示当前授权的用户。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'From': 发件人的邮箱地址。
    'Date': 邮件的发送日期。
    'Subject': 邮件的主题。
    'Snippet': 邮件的摘要（前100个字符）。
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')

    # 创建 Gmail API 客户端
    service = build('gmail', 'v1', credentials=creds)

    # 获取邮件列表
    results = service.users().messages().list(userId=user_id).execute()
    messages = results.get('messages', [])[:n]

    emails = []
    for message in messages:
        msg = service.users().messages().get(userId=user_id, id=message['id']).execute()

        # 解码邮件内容
        payload = msg['payload']
        headers = payload.get("headers")
        parts = payload.get("parts")

        data = {}

        if headers:
            for d in headers:
                name = d.get("name")
                if name.lower() == "from":
                    data['From'] = d.get("value")
                if name.lower() == "date":
                    data['Date'] = parser.parse(d.get("value")).strftime('%Y-%m-%d %H:%M:%S')
                if name.lower() == "subject":
                    data['Subject'] = d.get("value")

        if parts:
            for part in parts:
                mimeType = part.get("mimeType")
                body = part.get("body")
                data_decoded = base64.urlsafe_b64decode(body.get("data")).decode()
                if mimeType == "text/plain":
                    data['Snippet'] = data_decoded
                elif mimeType == "text/html":
                    soup = BeautifulSoup(data_decoded, "html.parser")
                    data['Snippet'] = soup.get_text()

        emails.append(data)

    # 返回邮件列表
    return json.dumps(emails, indent=2)

### 1.借助Few-shot完成需求函数的自动编写

&emsp;&emsp;在此前的内容中，我们已经成功的围绕某个具体的需求实现了让Chat模型自己编写代码完成需求。因此，现在距离让我们的智能邮件收发系统实现功能上的自生长就还剩下最后的关键一步，那就是将用户的实时需求进行翻译和整理，并形成一整套自动代码流程和提示方法，来引导模型进行对应的外部函数创建。这里我们假设在当前对话过程中，用户提出了一个全新的需求，那就是想要查看邮箱里是否有某位重要人物的未读邮件，并帮我解读邮件里面的内容。很明显我们此前定义的一系列外部函数都无法满足当前需求：

In [358]:
functions_list = [get_latest_email, send_email, retrieve_emails, get_email_counts]

In [359]:
functions = auto_functions(functions_list)

In [459]:
messages = [{"role": "user", "content": '请帮我查我的邮箱里是否有来自端木天的未读邮件，如果有的话请帮我解读下邮件内容。'}]

In [360]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  
    )

In [361]:
response

<OpenAIObject chat.completion id=chatcmpl-7fQGSV29Ve4krdVN7Xwz1M9veQDba at 0x2949e78af70> JSON: {
  "id": "chatcmpl-7fQGSV29Ve4krdVN7Xwz1M9veQDba",
  "object": "chat.completion",
  "created": 1690106764,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u4f5c\u4e3a\u4e00\u4e2aAI\uff0c\u6211\u6ca1\u6709\u6743\u9650\u8bbf\u95ee\u60a8\u7684\u4e2a\u4eba\u90ae\u7bb1\u3002\u6211\u5efa\u8bae\u60a8\u81ea\u5df1\u67e5\u770b\u90ae\u7bb1\uff0c\u6309\u7167\u90ae\u4ef6\u7684\u53d1\u9001\u8005\u6216\u8005\u5728\u641c\u7d22\u680f\u8f93\u5165\"\u9648\u660e\"\u641c\u7d22\u4ee5\u67e5\u627e\u76f8\u5173\u90ae\u4ef6\u3002\u8bb0\u5f97\u4fdd\u62a4\u597d\u60a8\u7684\u4e2a\u4eba\u4fe1\u606f\u54e6\u3002"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 27,
    "completion_tokens": 70,
    "total_tokens": 97
  }
}

In [362]:
response.choices[0].message['content']

'作为一个AI，我没有权限访问您的个人邮箱。我建议您自己查看邮箱，按照邮件的发送者或者在搜索栏输入"陈明"搜索以查找相关邮件。记得保护好您的个人信息哦。'

能够发现模型并没有找到能解决该问题外部函数，当前需求属于全新的需求。但从功能实现角度来说，这个功能和查询最近若干封邮件信息属于类似的功能，只要需求梳理到位、提示词合理，Chat模型应该是能按照要求编写实现该功能的函数的。

- 当前开发流程

&emsp;&emsp;当然，为了更加自动的完成这个新增功能的实现过程，我们需要一个提示流程，让Chat模型自行编写用于提示大语言模型创建外部函数的提示词，即我们需要编写一个用于创建提示词的提示词：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/5a74ea827ff69c59b4aec800caee481.png" alt="5a74ea827ff69c59b4aec800caee481" style="zoom:33%;" />

- Few-shot提示流程

&emsp;&emsp;仔细思考不难发现，这个提示词过程其实就是一个指令翻译过程，和我们在Ch.5中介绍的SCAN指令翻译任务类似。不过不同的是，SCAN指令翻译背后则是完全一套完全不同于自然语言的语法规则，而当前指令翻译任务相对简单，输入和输出都是可以基于自然语言的语义对其进行理解的，其背后的语法规则相对清晰。要完成这个指令翻译任务，让我们首先观察目前已有两个示例：

In [49]:
get_email_input = "请帮我查下邮箱里最后一封邮件内容。"

get_email_out = "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：\
                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                 3.请将全部功能封装在一个函数内；\
                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

email_counts_input = "请帮我查下邮箱里现在总共有多少封邮件。"

email_counts_out = "请帮我编写一个python函数，用于查看我的Gmail邮箱中总共有多少封邮件，函数要求如下：\
                    1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                    2.函数返回结果是当前邮件总数，返回结果本身必须是一个json格式对象；\
                    3.请将全部功能封装在一个函数内；\
                    4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

而在Ch.4和Ch.5中我们曾介绍，激发大语言模型的涌现能力进行指令翻译，最简单的方法是进行Few-shot。这里我们尝试先将上面两个例子作为提示示例，然后输入我们新的需求：“请帮我查下邮箱里是否有来自端木天的未读邮件，并解读最近一封未读邮件的内容。”试下Few-shot提示法是否足够引导模型完成当前任务，如果不行的话我们就继续尝试更高级的类似于CoT或者LtM提示法。这里Few-shot提示流程如下：

In [50]:
user_content = "请查下我的邮箱里是否有来自端木天的未读邮件，并解读最近一封未读邮件的内容"

messages_fewShot_stage1 = [{"role": "system", "content": "请按照格式，编写一段函数功能说明。"},
                          {"role": "user", "name":"example1_user", "content": get_email_input},
                          {"role": "assistant", "name":"example1_assistant", "content": get_email_out},
                          {"role": "user", "name":"example2_user", "content": email_counts_input},
                          {"role": "assistant", "name":"example2_assistant", "content": email_counts_out},
                          {"role": "user", "name":"example_user", "content": user_content}]
messages_fewShot_stage1

[{'role': 'system', 'content': '请按照格式，编写一段函数功能说明。'},
 {'role': 'user', 'name': 'example1_user', 'content': '请帮我查下邮箱里最后一封邮件内容。'},
 {'role': 'assistant',
  'name': 'example1_assistant',
  'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'user', 'name': 'example2_user', 'content': '请帮我查下邮箱里现在总共有多少封邮件。'},
 {'role': 'assistant',
  'name': 'example2_assistant',
  'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中总共有多少封邮件，函数要求如下：                    1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                    2.函数返回结果是当前邮件总数，返回结果本身必须是一个json格式对象；                    3.请将全部功能封装在一个函数内；                    4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'user',
  'name': 'example_user',
  'content': '请查下我的邮箱里是否有来自端木天

这里需要注意，尽管Chat模型是可以根据多轮的user-assistant实现Few-shot学习，但如果不在最开始的system message中设置输出要求是一段函数说明，模型会认为我们要他直接查看邮箱里面的邮件。因此这里的system message是一定需要编写的。然后测试提示效果：

In [43]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=messages_fewShot_stage1
)

In [46]:
response.choices[0].message['content']

'请帮我编写一个python函数，用于查看我的Gmail邮箱中是否有来自“端木天”的未读邮件，并解读最近一封未读邮件的内容，函数要求如下：\n\n1. 函数参数为userId，表示查看邮件对象，默认情况下取值为\'me\'，表示查看我的邮件；\n2. 如果有未读邮件，函数返回结果是一个包含最后一封未读邮件的信息和内容的对象，返回结果本身必须是一个json格式对象；如果没有未读邮件，返回{"status": "No unread mail from specified sender"}。\n3. 请将全部功能封装在一个函数内。\n4. 请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；'

通过上面返回的结果能够发现，模型能够较好的学习到Few-shot提示示例的指令翻译规范，并很好的完成了当前新的指令翻译工作。不过这里需要注意，该指令翻译结果其实还是存在一些潜在的问题，那就是无法区分“检查谁的邮箱”和“邮箱里和谁的通信”这两个对象，这或许是因为中文语境（“检查我的邮箱和端木天的通信邮件”）的多重表意带来的问题，但这确实是可能会影响到之后的代码编写。

&emsp;&emsp;接下来我们就将这段提示输入给Chat模型，令其根据这段提示创建查询函数。当然，为了确保效果的稳定性，我们仍然考虑采用Few-shot的方法来提示Chat模型完成代码编写，这里仍然以get_latest_email函数作为Few-shot提示示例，具体的提示过程如下：

In [15]:
system_content = "我现在已完成Gmail API授权，授权文件为本地文件token.json。"

In [16]:
user_example_content = "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：\
                        1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                        2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                        3.请将全部功能封装在一个函数内；\
                        4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [17]:
with open('./functions/tested functions%s_module.py' % 'get_latest_email', encoding='utf-8') as f:
    assistant_example_content = f.read()

In [18]:
assistant_example_content

'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的一封最新邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'messages\', [])\n\n    # 遍历邮件\n    for message in messages:\n        # 获取邮件的详细信息\n        msg = service.users().messages().get(userId=\'me\', id=message[\'id\']).execute()\n        \n    return json.dumps(msg)\n'

In [19]:
user_content = response.choices[0].message['content']

In [48]:
user_content

'请帮我编写一个python函数，用于查看我的Gmail邮箱中是否有来自“端木天”的未读邮件，并解读最近一封未读邮件的内容，函数要求如下：\n\n1. 函数参数为userId，表示查看邮件对象，默认情况下取值为\'me\'，表示查看我的邮件；\n2. 如果有未读邮件，函数返回结果是一个包含最后一封未读邮件的信息和内容的对象，返回结果本身必须是一个json格式对象；如果没有未读邮件，返回{"status": "No unread mail from specified sender"}。\n3. 请将全部功能封装在一个函数内。\n4. 请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；'

In [21]:
messages_fewShot_stage2 = [{"role": "system", "content": system_content},
                          {"role": "user", "name":"example_user", "content": user_example_content},
                          {"role": "assistant", "name":"example_assistant", "content": assistant_example_content},
                          {"role": "user", "name":"example_user", "content": user_content}]

查看此时Chat模型创建的函数：

In [22]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=messages_fewShot_stage2
)

In [54]:
s = response.choices[0].message['content']
s

'```python\nfrom googleapiclient.errors import HttpError\n\n def check_unread_email_from_DuanMuTian(userId=\'me\'):\n    """\n    查询Gmail邮箱中是否有来自端木天未读邮件\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    需要注意的是当查询我的邮箱时，userId需要输入\'me\'\n    :return: 返回一个json格式对象，指示是否存在来自端木天的未读邮件\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)  \n    \n    # 查询是否有来自端木天的未读邮件\n    query = "is:unread from:{userId}"\n    results = service.users().messages().list(userId=userId, q=query).execute()\n    messages = results.get(\'messages\', [])\n\n    # 如果uid列表不为空，则表示存在未读邮件\n    if messages:\n        return json.dumps({"exist_unread_email": True})\n    else:\n        return json.dumps({"exist_unread_email": False}) \n```\n在这个函数中，我使用了`service.users().messages().list`这个Gmail API的方法来获取邮件列表，根据是否含有特定名字的未读邮件进行筛选。如果结果列表不为空，则说明有未读邮件，否则说明没有。然后使用`service.users().messages().get`方法获取邮件详情，

然后将其提取为本地代码文件：

In [60]:
extract_function_code(s, detail=1)

def check_unread_email_from_DuanMuTian(userId='me'):
    """
    查询Gmail邮箱中是否有来自端木天未读邮件
    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\
    需要注意的是当查询我的邮箱时，userId需要输入'me'
    :return: 返回一个json格式对象，指示是否存在来自端木天的未读邮件
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')
    
    # 创建 Gmail API 客户端
    service = build('gmail', 'v1', credentials=creds)  
    
    # 查询是否有来自端木天的未读邮件
    query = "is:unread from:{userId}"
    results = service.users().messages().list(userId=userId, q=query).execute()
    messages = results.get('messages', [])

    # 如果uid列表不为空，则表示存在未读邮件
    if messages:
        return json.dumps({"exist_unread_email": True})
    else:
        return json.dumps({"exist_unread_email": False}) 



该函数看起来“有模有样”，但是，此前提示存在的问题在代码环节先露无遗，尽管代码的说明是提取和端木天有关的未读邮件，但只有一个UserId参数，从代码说明来看，这个参数或许应该是查询发送邮件的对象，但实际代码执行时候这个查询对象是自己，也就是说，这个函数实际上只能用于查询自己和自己通信的未读邮件，如果UserId是me，则查询的是Gmail和自己通信的未读邮件，而如果UserId是端木天，则查询的是端木天和自己的通信未读邮件。因此这个代码尽管可用，但并不能达到我们预期的效果。

&emsp;&emsp;当然，围绕上面的问题，究其原因还是因为用于提示创建代码的提示语句有问题，容易造成误解。而容易造成误解的根本原因则是第一阶段在创建这段提示语句时，并没有创建得到合理的关于函数参数的描述——一个更加合理的参数描述应该是设置两个参数，分别用于表示“查询谁的邮箱”、以及“查询和谁的通信”。

### 2.借助LtM提示流程完成更稳定的函数编写

- LtM提示过程

&emsp;&emsp;而如果在Few-shot的提示下，第一阶段的指令翻译问题无法输出令人满意的结果，首先我们想到的解决问题的办法就是提供一些更加强有力的提示方法：例如LtM提示法。根据LtM多段提示的策略，外加结合当前指令翻译任务考虑，一种更有希望能够引导模型能够顺利完成指令翻译流程的提示方法是：先引导模型拆解当前需求中的“变量”作为后续函数的参数，然后再基于已经确定的变量，引导模型完成翻译工作。其实就整个指令翻译任务来说，最难的地方也就在于函数参数的理解过程，而一旦模型能够准确的翻译出外部函数需要哪些参数，其他部分的翻译任务自然迎刃而解。即在修改了第一阶段提示流程后，现在整体提示过程如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/0fe2f6a0d89760b9b2d7811577249b7.png" alt="0fe2f6a0d89760b9b2d7811577249b7" style="zoom:33%;" />

> 此外，需要注意的是，让模型合理的分析符合当前需求的参数设置，其实也是一种提高函数复用率的措施。例如现在想查询来自端木天的未读邮件，若模型可以将读取“谁”的未读邮件视作一个变量，即将读取邮件的对象设置为参数，则可以更好的提升后续该函数的复用率。

&emsp;&emsp;对于LtM提示的两个阶段，我们沿用Ch.5指令翻译相关内容中的命名方法，即第一个阶段的提示命名为Command decomposition，简写为CD，第二阶段的提示命名为Command mapping，简写为CM。并且。需要注意的是，我们这里稍微调整了下Few-shot的提示示例，这里我们以查看最后一封邮件内容和发送邮件两个函数作为LtM提示的两个阶段的示例，具体提示流程如下：

> 需要注意，根据Ch.5的内容介绍，LtM两个阶段的提示示例其实可以不同，目前由于我们也并没有存储太多示例函数，因此这里LtM两个阶段的提示都采用相同的示例。

In [61]:
system_content1 = "为了更好编写满足用户需求的python函数，我们需要先识别用户需求中的变量，以作为python函数的参数。需要注意的是，当前编写的函数中涉及到的邮件收发查阅等功能，都是通过调用Gmail API来完成。"

In [68]:
input1 = "请帮我查下Gmail邮箱里最后一封邮件内容。"

In [69]:
pi1 = "当前需求中可以作为函数参数的是：1.查看谁的邮箱。"

In [70]:
input2 = "请帮我给陈明发送一封Gmail邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。"

In [71]:
pi2 = "当前需求中可以作为函数参数的是：1.发送邮件的对象；2.发送邮件的主题；3.邮件具体内容"

In [72]:
input3 = "请查下我的邮箱里是否有来自端木天的未读邮件，并解读最近一封未读邮件的内容。"

In [534]:
messages_CD = [{"role": "system", "content": system_content1},
                {"role": "user", "name":"example1_user", "content": input1},
                {"role": "assistant", "name":"example1_assistant", "content": pi1},
                {"role": "user", "name":"example2_user", "content": input2},
                {"role": "assistant", "name":"example2_assistant", "content": pi2},
                {"role": "user", "name":"example_user", "content": input3}]

In [535]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=messages_CD
)

In [536]:
pi3 = response.choices[0].message['content']

In [537]:
pi3

'当前需求中可以作为函数参数的是：1.查询谁的邮箱；2.查询的邮件来源者'

能够发现，当我们将问题拆解并在第一个阶段给与提示，模型能够非常清楚的判断当前需求中可以作为函数的变量。

- LtM Command mapping阶段提示

&emsp;&emsp;接下来继续进行LtM第二阶段提示，并最终输出合理的函数创建提示语句：

In [73]:
system_content2 = "我现在已完成Gmail API授权，授权文件为本地文件token.json。所编写的函数要求参数类型都必须是字符串类型。"

In [74]:
get_email_input = "请帮我查下邮箱里最后一封邮件内容。" + pi1
get_email_input

'请帮我查下邮箱里最后一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'

In [75]:
get_email_out = "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：\
                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                 3.请将全部功能封装在一个函数内；\
                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [76]:
send_email_input = "请帮我给陈明发送一封Gmail邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。" + pi2
send_email_input

'请帮我给陈明发送一封Gmail邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。当前需求中可以作为函数参数的是：1.发送邮件的对象；2.发送邮件的主题；3.邮件具体内容'

In [77]:
send_email_out = "请帮我编写一个python函数，用于给陈明发送邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划，函数要求如下：\
                  1.函数参数to、subject和message_text，三个参数都是字符串类型，其中to表示发送邮件对象，subject表示邮件主题，message_text表示邮件具体内容；\
                  2.函数返回结果是当前邮件发送状态，返回结果本身必须是一个json格式对象；\
                  3.请将全部功能封装在一个函数内；\
                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [67]:
user_content = input3 + pi3
user_content

'请查下我的邮箱里是否有来自端木天的未读邮件，并解读最近一封未读邮件的内容。当前需求中可以作为函数参数的是：1.查询谁的邮箱；2.查询的邮件来源者'

In [78]:
messages_CM = [{"role": "system", "content": system_content2},
                   {"role": "user", "name":"example1_user", "content": get_email_input},
                   {"role": "assistant", "name":"example1_assistant", "content": get_email_out},
                   {"role": "user", "name":"example2_user", "content": send_email_input},
                   {"role": "assistant", "name":"example2_assistant", "content": send_email_out},
                   {"role": "user", "name":"example_user", "content": user_content}]

In [79]:
messages_CM

[{'role': 'system',
  'content': '我现在已完成Gmail API授权，授权文件为本地文件token.json。所编写的函数要求参数类型都必须是字符串类型。'},
 {'role': 'user',
  'name': 'example1_user',
  'content': '请帮我查下邮箱里最后一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
 {'role': 'assistant',
  'name': 'example1_assistant',
  'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'user',
  'name': 'example2_user',
  'content': '请帮我给陈明发送一封Gmail邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。当前需求中可以作为函数参数的是：1.发送邮件的对象；2.发送邮件的主题；3.邮件具体内容'},
 {'role': 'assistant',
  'name': 'example2_assistant',
  'content': '请帮我编写一个python函数，用于给陈明发送邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划，函数要求如下：                  1.函数参数to、subject和message_text，三个参数都是字符串类型，其中to表示发送邮件对象，subject表示邮件主题，message_text表示邮件具体内容；                  2.函数返回结果是当前邮件发送状态，返回结果本身必须是一

In [80]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=messages_CM
)

查看此时Chat模型创建函数描述对象：

In [81]:
function_description = response.choices[0].message['content']
function_description

"请帮我编写一个python函数，用于查看我的Gmail邮箱中是否有来自端木天的未读邮件，并将最近一封未读邮件的内容解读出来，函数要求如下：                 1.函数参数userId和query，userId和query参数均是字符串类型，其中userId默认值为'me'，表示查看我的邮件，query参数表示查询的邮件来源者；                 2.函数返回结果是一个包含查询邮件内容的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

能够发现，在合理的函数参数设置下，这段函数描述看起来也更加合理。至此我们就完成了基于LtM的函数描述创建的提示过程。接下来我们继续进行整体提示的第二阶段，即引导大模型基于function_description进行相应函数的创建工作。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/44740c7f86aa64ce43b217fb9b000fb.png" alt="44740c7f86aa64ce43b217fb9b000fb" style="zoom:33%;" />

- 函数创建

&emsp;&emsp;在函数创建阶段，我们同样采用Few-shot的方式进行提示，提示的示例仍然为get_email_out和send_email_out两个函数，提示流程如下：

In [82]:
func1_description = get_email_out
func1_description

"请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [83]:
with open('./functions/tested functions/%s_module.py' % 'get_latest_email', encoding='utf-8') as f:
    func1_str = f.read()

In [84]:
func1_str

'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的一封最新邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'messages\', [])\n\n    # 遍历邮件\n    for message in messages:\n        # 获取邮件的详细信息\n        msg = service.users().messages().get(userId=\'me\', id=message[\'id\']).execute()\n        \n    return json.dumps(msg)\n'

In [85]:
func2_description = send_email_out
func2_description

'请帮我编写一个python函数，用于给陈明发送邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划，函数要求如下：                  1.函数参数to、subject和message_text，三个参数都是字符串类型，其中to表示发送邮件对象，subject表示邮件主题，message_text表示邮件具体内容；                  2.函数返回结果是当前邮件发送状态，返回结果本身必须是一个json格式对象；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；'

In [86]:
with open('./functions/tested functions/%s_module.py' % 'send_email', encoding='utf-8') as f:
    func2_str = f.read()

In [87]:
func2_str

'def send_email(to, subject, message_text):\n    """\n    借助Gmail API创建并发送邮件函数\n    :param to: 必要参数，字符串类型，用于表示邮件发送的目标邮箱地址；\n    :param subject: 必要参数，字符串类型，表示邮件主题；\n    :param message_text: 必要参数，字符串类型，表示邮件全部正文；\n    :return：返回发送结果字典，若成功发送，则返回包含邮件ID和发送状态的字典。\n    """\n    \n    creds_file=\'token_send.json\'\n    \n    def create_message(to, subject, message_text):\n        """创建一个MIME邮件"""\n        message = MIMEText(message_text)\n        message[\'to\'] = to\n        message[\'from\'] = \'me\'\n        message[\'subject\'] = subject\n        raw_message = base64.urlsafe_b64encode(message.as_string().encode(\'utf-8\')).decode(\'utf-8\')\n        return {\n            \'raw\': raw_message\n        }\n\n    def send_message(service, user_id, message):\n        """发送邮件"""\n        try:\n            sent_message = service.users().messages().send(userId=user_id, body=message).execute()\n            print(f\'Message Id: {sent_message["id"]}\')\n            return sent_message\n        except

In [88]:
system_content3 = "我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。"

In [90]:
function_description

"请帮我编写一个python函数，用于查看我的Gmail邮箱中是否有来自端木天的未读邮件，并将最近一封未读邮件的内容解读出来，函数要求如下：                 1.函数参数userId和query，userId和query参数均是字符串类型，其中userId默认值为'me'，表示查看我的邮件，query参数表示查询的邮件来源者；                 2.函数返回结果是一个包含查询邮件内容的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [109]:
messages_stage2 = [{"role": "system", "content": system_content3},
                    {"role": "user", "name":"example1_user", "content": func1_description},
                    {"role": "assistant", "name":"example1_assistant", "content": func1_str},
                    {"role": "user", "name":"example2_user", "content": func2_description},
                    {"role": "assistant", "name":"example2_assistant", "content": func2_str},
                    {"role": "user","content": function_description}]

In [110]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=messages_stage2
)

In [111]:
s = response.choices[0].message['content']
s

'def get_unread_email(userId=\'me\', query=\'\'):\n    """\n    查询Gmail邮箱中是否有来自特定发件人的未读邮件，并解读最近的一封未读邮件内容。\n    \n    :param userId: 可选参数，默认值为 \'me\'，表示查看我的邮件。userId是用户的邮件地址。 \n    :param query: 可选参数，字符串类型，表示查询的邮件来源者。邮件来源者的名字或者邮箱地址都可以写在这里。\n    :return: 返回一个json格式的字符串，该字符串包含了查询到的邮件内容，如果没有查询到符合条件的邮件，就返回一个空的json对象字符串。\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n\n    # 格式化我们的查询\n    query = f"from:({query}) is:unread"\n    \n    # 使用 Gmail API 中的 list 方法列出满足我们查询条件的前 1 封邮件\n    results = service.users().messages().list(\n        userId=userId, \n        maxResults=1, \n        q=query).execute()\n    messages = results.get(\'messages\', [])\n\n    # 如果邮件存在\n    if messages:\n        for message in messages:\n            # 获取邮件的详细信息\n            msg = service.users().messages().get(userId=userId, id=message[\'id\']).execute()\n            \n        retu

接下来我们将字符串解析成对应的函数并进行运行：

In [112]:
extract_function_code(s, detail=1)

def get_unread_email(userId='me', query=''):
    """
    查询Gmail邮箱中是否有来自特定发件人的未读邮件，并解读最近的一封未读邮件内容。
    
    :param userId: 可选参数，默认值为 'me'，表示查看我的邮件。userId是用户的邮件地址。 
    :param query: 可选参数，字符串类型，表示查询的邮件来源者。邮件来源者的名字或者邮箱地址都可以写在这里。
    :return: 返回一个json格式的字符串，该字符串包含了查询到的邮件内容，如果没有查询到符合条件的邮件，就返回一个空的json对象字符串。
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')
    
    # 创建 Gmail API 客户端
    service = build('gmail', 'v1', credentials=creds)

    # 格式化我们的查询
    query = f"from:({query}) is:unread"
    
    # 使用 Gmail API 中的 list 方法列出满足我们查询条件的前 1 封邮件
    results = service.users().messages().list(
        userId=userId, 
        maxResults=1, 
        q=query).execute()
    messages = results.get('messages', [])

    # 如果邮件存在
    if messages:
        for message in messages:
            # 获取邮件的详细信息
            msg = service.users().messages().get(userId=userId, id=message['id']).execute()
            
        return json.dumps(msg)
    else:
        return jso

此时模型定义的函数为get_unread_email，通过人工审核应该能发现，该函数能够满足当前功能需求。接下来我们尝试带入该函数到Chat模型中进行功能验证：

In [114]:
get_unread_email

<function __main__.get_unread_email(userId='me', query='')>

In [116]:
functions_list = [get_unread_email]

In [121]:
functions = auto_functions(functions_list)

In [122]:
messages = [{"role": "user", "content": '请查询我的邮箱，看下里面是否有来自端木天的未读邮件。'}]

In [123]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  
    )

In [124]:
response

<OpenAIObject chat.completion id=chatcmpl-7flOTK1poEcdoyElD2OLuJW4jnror at 0x1e1f9bc84a0> JSON: {
  "id": "chatcmpl-7flOTK1poEcdoyElD2OLuJW4jnror",
  "object": "chat.completion",
  "created": 1690187985,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_unread_email",
          "arguments": "{\n  \"userId\": \"me\",\n  \"query\": \"\u7aef\u6728\u5929\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 126,
    "completion_tokens": 26,
    "total_tokens": 152
  }
}

能够发现，模型能够正常使用用户需求，并创建对应的符合要求的参数。接下来将该函数直接带入到多轮对话函数中测试效果，并同时带入其他外部函数以便进行更加复杂的功能测试。这里我们提前往Gmail邮箱中发送一封邮件，邮件内容如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20230724164749799.png" alt="image-20230724164749799" style="zoom:20%;" />

接下来进行功能测试：

In [571]:
functions_list = [get_latest_email, send_email, retrieve_emails, get_email_counts, get_unread_email]

In [574]:
chat_with_model(functions_list=functions_list, 
                system_message=[{"role": "system", "content": "端木天的邮箱地址是:2323365771@qq.com"}])

模型回答: 你好！有什么可以帮助您的吗？


您还有其他问题吗？(输入退出以结束对话):  我的邮箱里有来自端木天的未读邮件么？


模型回答: 是的，您有一封来自端木天的未读邮件。这封邮件的主题是“本周工作进度”，他在邮件中提到：“下半年技术开发计划已做好，随时可以向您进行汇报。”


您还有其他问题吗？(输入退出以结束对话):  好的，请给他回复邮件，请他明天上午10点来我的办公室进行下半年开发计划的汇报。


Message Id: 18982749d8f872e9
模型回答: 好的，已经给端木天发送了邮件，通知他明天上午10点来你的办公室进行下半年开发计划的汇报。


您还有其他问题吗？(输入退出以结束对话):  退出


而对话完成后，我们即可在端木天的邮箱中查看到回复邮件如下，注意这个邮件回复的过程实际上是调用了send_email功能来进行的实现：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/c021d422fdc861cd6467821fe797eec.jpg" alt="c021d422fdc861cd6467821fe797eec" style="zoom:20%;" />

至此，我们就完整的执行了一次新需求自动编写函数实现的完整流程。

### 3.全自动功能函数编写流程

&emsp;&emsp;接下来，我们将上述基于LtM提示流程的稳定编写外部函数流程进行更高层次的函数封装，使得能够更加自动化进行外部函数创建。这里需要注意的是，在进行高层次函数封装之前，我们需要围绕上述全自动开发流程进行更高层次的抽象，以获得一个更为一般的、适用于更多一般情况的全自动开发流程。

#### 3.1 提示词管理

&emsp;&emsp;首先，面对一个基于大语言模型的全自动的代码编写和功能开发流程，非常重要的一点是对各模型、各环节的提示词进行妥善管理，以便提升对大语言模型的掌控力。当然，不同开发项目在前期需要探索得到适用于当前开发流程的（能获得稳定代码结果的）提示流程，例如对于当前开发项目而言，总共是两个阶段进行提示，并且第一阶段还会采用LtM提示法，因此每个函数的背后总共都有三个提示阶段，这里我们可以围绕每个经过测试的函数，创建一个JSON对象以保存其每个阶段的提示过程。

> 传统开发项目的代码管理有多重要，全自动开发流程的提示词管理就有多重要。

- get_latest_email完整提示示例创建过程

&emsp;&emsp;首先是get_latest_email函数各阶段提示词及提示结果保存过程。在此前的提示过程中，我们以该函数作为提示示例，非常好的完成了Few-shot提示过程。此前的流程中我们手动编写的该函数的三个阶段提示词及结果如下：

In [3]:
# 第一阶段LtM_CD阶段提示词及输出结果
get_latest_email_CD_input = "请帮我查下Gmail邮箱里最后一封邮件内容。"
get_latest_email_pi = "当前需求中可以作为函数参数的是：1.查看谁的邮箱。"
get_latest_email_messages_CD = [
                                {"role": "user", "content": get_latest_email_CD_input},
                                {"role": "assistant", "content": get_latest_email_pi}
                                ]

# 第一阶段LtM_CM阶段提示词及输出结果
get_latest_email_CM_input = get_latest_email_CD_input + get_latest_email_pi
get_latest_email_description = "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：\
                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                 3.请将全部功能封装在一个函数内；\
                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"
get_latest_email_messages_CM = [
                                {"role": "user", "content": get_latest_email_CM_input},
                                {"role": "assistant", "content":get_latest_email_description}
                                ]

# 第二阶段提示词及输出结果
with open('./functions/tested functions/%s_module.py' % 'get_latest_email', encoding='utf-8') as f:
    get_latest_email_function = f.read()
get_latest_email_messages = [
                             {"role": "user", "content": get_latest_email_description},
                             {"role": "assistant", "content":get_latest_email_function}
                             ]  

这里有两点需要注意，其一是在提示词保存过程中，不仅需要保存提示的提示词，还需要保存提示结果，以便作为其他提示过程的提示示例；其二则是保存形式方面，考虑到实际开发过程多用Chat模型，因此建议以类messages参数形式进行保存，而上面的代码其实就是根据提示的各个阶段，分阶段保存每个阶段的示例messages（即一个role message、一个assistant message）。

&emsp;&emsp;在以message形式保存完每个阶段的提示内容之后，接下来我们将三个提示示例以如下格式进行保存，即将一个函数对象的三个阶段的全部提示示例保存为一个字典，并且字典中的每个键值对代表一个阶段的提示内容。同时，每个函数的完整提示示例都以function_name_prompt命名，具体get_latest_email_prompt对象创建过程如下：

In [4]:
get_latest_email_prompt = {
                            "stage1_CD": get_latest_email_messages_CD,
                            "stage1_CM": get_latest_email_messages_CM,
                            "stage2": get_latest_email_messages
                          }    

In [5]:
get_latest_email_prompt

{'stage1_CD': [{'role': 'user', 'content': '请帮我查下Gmail邮箱里最后一封邮件内容。'},
  {'role': 'assistant', 'content': '当前需求中可以作为函数参数的是：1.查看谁的邮箱。'}],
 'stage1_CM': [{'role': 'user',
   'content': '请帮我查下Gmail邮箱里最后一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
  {'role': 'assistant',
   'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"}],
 'stage2': [{'role': 'user',
   'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
  {'role': 'assistant',
   'content': 'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n 

接下来即可将其写入本地，并将其和对应函数的py文件保存为同一文件夹内：

In [6]:
with open('./functions/tested functions/%s_prompt.json' % 'get_latest_email', 'w') as f:
    json.dump(get_latest_email_prompt, f)

而在保存了完整的提示内容（提示示例）之后，之后可以采用如下方式进行读取：

In [7]:
with open('./functions/tested functions/%s_prompt.json' % 'get_latest_email', 'r') as f:
    data = json.load(f)

In [8]:
data

{'stage1_CD': [{'role': 'user', 'content': '请帮我查下Gmail邮箱里最后一封邮件内容。'},
  {'role': 'assistant', 'content': '当前需求中可以作为函数参数的是：1.查看谁的邮箱。'}],
 'stage1_CM': [{'role': 'user',
   'content': '请帮我查下Gmail邮箱里最后一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
  {'role': 'assistant',
   'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"}],
 'stage2': [{'role': 'user',
   'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
  {'role': 'assistant',
   'content': 'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n 

- send_email、retrieve_emails和get_email_counts完整提示示例创建过程

&emsp;&emsp;接下来我们继续将此前定义好的send_email、retrieve_emails和get_email_counts函数相关提示流程进行本地保存。首先send_email相关提示示例整理如下：

In [9]:
send_email_CD_input = "请帮我给陈明发送一封Gmail邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。"
send_email_pi = "当前需求中可以作为函数参数的是：1.发送邮件的对象；2.发送邮件的主题；3.邮件具体内容"
send_email_messages_CD = [
                          {"role": "user", "content": send_email_CD_input},
                          {"role": "assistant", "content": send_email_pi}
                         ]

send_email_CM_input = send_email_CD_input + send_email_pi
send_email_description = "请帮我编写一个python函数，用于给陈明发送邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划，函数要求如下：\
                  1.函数参数to、subject和message_text，三个参数都是字符串类型，其中to表示发送邮件对象，subject表示邮件主题，message_text表示邮件具体内容；\
                  2.函数返回结果是当前邮件发送状态，返回结果本身必须是一个json格式对象；\
                  3.请将全部功能封装在一个函数内；\
                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"
send_email_messages_CM = [
                          {"role": "user", "content": send_email_CM_input},
                          {"role": "assistant", "content":send_email_description}
                         ]

with open('./functions/tested functions/%s_module.py' % 'send_email', encoding='utf-8') as f:
    send_email_function = f.read()
send_email_messages = [
                       {"role": "user", "content": send_email_description},
                       {"role": "assistant", "content":send_email_function}
                      ] 

完整的包含全部提示示例的send_email_prompt对象如下：

In [10]:
send_email_prompt = {
                     "stage1_CD": send_email_messages_CD,
                     "stage1_CM": send_email_messages_CM,
                     "stage2": send_email_messages
                    }       

同样的，将其写入send_email_prompt.json文件中：

In [11]:
with open('./functions/tested functions/%s_prompt.json' % 'send_email', 'w') as f:
    json.dump(send_email_prompt, f)

&emsp;&emsp;然后是retrieve_emails和get_email_counts函数的提示示例编写过程。需要注意的是，该函数是上一小节借助ChatGPT和Chat模型编写的两个函数，这两个函数此前并没有编写完整的三阶段提示词，这里需要重新进行编写。首先关于retrieve_emails函数三阶段提示示例编写如下：

In [12]:
code_retrieve_emails = inspect.getsource(retrieve_emails)
with open('./functions/tested functions/%s_module.py' % 'retrieve_emails', 'w', encoding='utf-8') as f:
    f.write(code_retrieve_emails)

In [13]:
retrieve_emails_CD_input = "请帮我查看下最近5封邮件的邮件内容"
retrieve_emails_pi = "当前需求中可以作为函数参数的是：1.查询几封邮件；2.查询谁的邮箱"
retrieve_emails_messages_CD = [
                          {"role": "user", "content": retrieve_emails_CD_input},
                          {"role": "assistant", "content": retrieve_emails_pi}
                         ]

retrieve_emails_CM_input = retrieve_emails_CD_input + retrieve_emails_pi
retrieve_emails_description = "请帮我编写一个python函数，查询我的邮箱里面最近的几封邮件信息，函数要求如下：\
                  1.函数参数n和user_id，两个参数都是字符串类型，其中n表示查询最近的几封邮件，user_id表示要检索邮件的用户的ID。默认值是'me'，表示当前授权的用户；\
                  2.函数返回结果是查询到的这几封邮件的邮件内容，返回结果本身必须是一个json格式对象；\
                  3.请将全部功能封装在一个函数内；\
                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"
retrieve_emails_messages_CM = [
                          {"role": "user", "content": retrieve_emails_CM_input},
                          {"role": "assistant", "content":retrieve_emails_description}
                         ]

with open('./functions/tested functions/%s_module.py' % 'retrieve_emails', encoding='utf-8') as f:
    retrieve_emails_function = f.read()
retrieve_emails_messages = [
                       {"role": "user", "content": retrieve_emails_description},
                       {"role": "assistant", "content":retrieve_emails_function}
                      ] 

In [14]:
retrieve_emails_prompt = {
                     "stage1_CD": retrieve_emails_messages_CD,
                     "stage1_CM": retrieve_emails_messages_CM,
                     "stage2": retrieve_emails_messages
                    }   

In [15]:
retrieve_emails_prompt

{'stage1_CD': [{'role': 'user', 'content': '请帮我查看下最近5封邮件的邮件内容'},
  {'role': 'assistant', 'content': '当前需求中可以作为函数参数的是：1.查询几封邮件；2.查询谁的邮箱'}],
 'stage1_CM': [{'role': 'user',
   'content': '请帮我查看下最近5封邮件的邮件内容当前需求中可以作为函数参数的是：1.查询几封邮件；2.查询谁的邮箱'},
  {'role': 'assistant',
   'content': "请帮我编写一个python函数，查询我的邮箱里面最近的几封邮件信息，函数要求如下：                  1.函数参数n和user_id，两个参数都是字符串类型，其中n表示查询最近的几封邮件，user_id表示要检索邮件的用户的ID。默认值是'me'，表示当前授权的用户；                  2.函数返回结果是查询到的这几封邮件的邮件内容，返回结果本身必须是一个json格式对象；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"}],
 'stage2': [{'role': 'user',
   'content': "请帮我编写一个python函数，查询我的邮箱里面最近的几封邮件信息，函数要求如下：                  1.函数参数n和user_id，两个参数都是字符串类型，其中n表示查询最近的几封邮件，user_id表示要检索邮件的用户的ID。默认值是'me'，表示当前授权的用户；                  2.函数返回结果是查询到的这几封邮件的邮件内容，返回结果本身必须是一个json格式对象；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
  {'role': 'assistant',
   'cont

In [16]:
with open('./functions/tested functions/%s_prompt.json' % 'retrieve_emails', 'w') as f:
    json.dump(retrieve_emails_prompt, f)

然后get_email_counts函数三个阶段提示示例如下：

In [17]:
get_email_counts_CD_input = "请帮我查下邮箱里面总共有多少封邮件。"
get_email_counts_pi = "当前需求中可以作为函数参数的是：1.查询谁的邮箱"
get_email_counts_messages_CD = [
                          {"role": "user", "content": get_email_counts_CD_input},
                          {"role": "assistant", "content": get_email_counts_pi}
                         ]

get_email_counts_CM_input = get_email_counts_CD_input + get_email_counts_pi
get_email_counts_description = "请帮我编写一个python函数，用于查看我的Gmail邮箱中总共有多少封邮件，函数要求如下：\
                  1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                  2.函数返回结果是查询到的这几封邮件的邮件内容，返回结果本身必须是一个json格式对象；\
                  3.请将全部功能封装在一个函数内；\
                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"
get_email_counts_messages_CM = [
                          {"role": "user", "content": get_email_counts_CM_input},
                          {"role": "assistant", "content":get_email_counts_description}
                         ]

with open('./functions/tested functions/%s_module.py' % 'get_email_counts', encoding='utf-8') as f:
    get_email_counts_function = f.read()
get_email_counts_messages = [
                       {"role": "user", "content": get_email_counts_description},
                       {"role": "assistant", "content":get_email_counts_function}
                      ] 

In [18]:
get_email_counts_prompt = {
                     "stage1_CD": get_email_counts_messages_CD,
                     "stage1_CM": get_email_counts_messages_CM,
                     "stage2": get_email_counts_messages
                    }   

In [19]:
get_email_counts_prompt

{'stage1_CD': [{'role': 'user', 'content': '请帮我查下邮箱里面总共有多少封邮件。'},
  {'role': 'assistant', 'content': '当前需求中可以作为函数参数的是：1.查询谁的邮箱'}],
 'stage1_CM': [{'role': 'user',
   'content': '请帮我查下邮箱里面总共有多少封邮件。当前需求中可以作为函数参数的是：1.查询谁的邮箱'},
  {'role': 'assistant',
   'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中总共有多少封邮件，函数要求如下：                  1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                  2.函数返回结果是查询到的这几封邮件的邮件内容，返回结果本身必须是一个json格式对象；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"}],
 'stage2': [{'role': 'user',
   'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中总共有多少封邮件，函数要求如下：                  1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                  2.函数返回结果是查询到的这几封邮件的邮件内容，返回结果本身必须是一个json格式对象；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
  {'role': 'assistant',
   'content': 'def get_email_counts(userId=\'me\'):\n    """\n    查询Gmail邮箱中邮件总数\n  

In [21]:
with open('./functions/tested functions/%s_prompt.json' % 'get_email_counts', 'w') as f:
    json.dump(get_email_counts_prompt, f)

至此，我们就完成了此前或自己手动编写、或引导Chat模型进行编写的函数全部提示示例的编写和保存。

- 系统提示消息保存

&emsp;&emsp;当然，对于我们当前提示流程中各阶段的系统提示消息，也需要对其进行整理和本地保存：

In [25]:
system_messages = {"system_message_CD": [{"role": "system", "content": "为了更好编写满足用户需求的python函数，我们需要先识别用户需求中的变量，以作为python函数的参数。需要注意的是，当前编写的函数中涉及到的邮件收发查阅等功能，都是通过调用Gmail API来完成。"}], 
                   "system_message_CM": [{"role": "system", "content": "我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。"}], 
                   "system_message": [{"role": "system", "content":"我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。"}]}

In [26]:
with open('./functions/tested functions/%s.json' % 'system_messages', 'w') as f:
    json.dump(system_messages, f)

#### 3.2 项目代码文件管理

&emsp;&emsp;截止目前，我们已经围绕当前智能邮件项目创建了非常多的函数代码和相关提示词说明文档，而为了之后更好的对当前项目进行管理，我们有必要对当前项目的代码结构和文件结构进行修改。当然，我们当前的AI项目主要是围绕着外部函数的功能拓展这部分进行开发，整体项目代码管理的复杂程度并不高。

&emsp;&emsp;这里首先需要对本地保存的文件及文件夹路径进行调整。为了更方便的借助本地文件系统进行代码和提示词的管理，我们需要将原先的functions文件夹更名为autoGmail_project，以注明是该文件夹专门用于保存autoGmail相关文件。并且我们需要将tested和untested内的同名函数代码和提示文件保存到一个同名文件夹内，而原system_messages.json文件位置不变。修改前tested文件夹内文件情况如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/0ce947634db31f8fc32ccaabb5a2370.png" alt="0ce947634db31f8fc32ccaabb5a2370" style="zoom:33%;" />

修改后，我们将同一个函数的module文件和prompt文件都放在一个函数同名文件夹内，修改后tested文件如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/1797fa58878a81564e80aebb0878641.png" alt="1797fa58878a81564e80aebb0878641" style="zoom:33%;" />

#### 3.3 提示词管理辅助函数

&emsp;&emsp;最后需要注意的是，由于接下来是以文件夹形式进行代码和提示词的管理，因此我们需要重新定义一个remove_to_tested函数，以实现函数文件夹的untested到tested的转移过程：

In [76]:
def remove_to_tested(function_name):
    """
    将函数同名文件夹由untested文件夹转移至tested文件夹内。\
    完成转移则说明函数通过测试，可以使用。此时需要将该函数的源码写入gptLearning.py中方便下次调用。
    """
    
    # 将函数代码写入gptLearning.py文件中
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), encoding='utf-8') as f:
        function_code = f.read()
    
    with open('gptLearning.py', 'a', encoding='utf-8') as f:
        f.write(function_code)
    
    # 源文件夹路径
    src_dir = './functions/untested functions/%s' % function_name

    # 目标文件夹路径
    dst_dir = './functions/tested functions/%s' % function_name
    
    # 移动文件夹
    shutil.move(src_dir, dst_dir)

同时，此前定义的extract_function_code和show_functions这两个函数也需要进行修改，其中extract_function_code需要添加在完成函数提取时就在untested文件夹中创建同名函数文件夹的功能，以及需要求修改读取和写入代码的文件路径：

In [46]:
def extract_function_code(s, detail=0, tested=False, g=globals()):
    """
    函数提取函数，同时执行函数内容，可以选择打印函数信息，并选择代码保存的地址
    """
    def extract_code(s):
        """
        如果输入的字符串s是一个包含Python代码的Markdown格式字符串，提取出代码部分。
        否则，返回原字符串。

        参数:
        s: 输入的字符串。

        返回:
        提取出的代码部分，或原字符串。
        """
        # 判断字符串是否是Markdown格式
        if '```python' in s or 'Python' in s or'PYTHON' in s:
            # 找到代码块的开始和结束位置
            code_start = s.find('def')
            code_end = s.find('```\n', code_start)
            # 提取代码部分
            code = s[code_start:code_end]
        else:
            # 如果字符串不是Markdown格式，返回原字符串
            code = s

        return code
    
    # 提取代码字符串
    code = extract_code(s)
    
    # 提取函数名称
    match = re.search(r'def (\w+)', code)
    function_name = match.group(1)
    
    # 在untested文件夹内创建函数同名文件夹
    directory = './functions/untested functions/%s' % function_name
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # 将函数写入本地
    if tested == False:
        with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), 'w', encoding='utf-8') as f:
            f.write(code)
    else:
        # 调用remove_to_test函数将函数文件夹转移至tested文件夹内
        remove_to_tested(function_name)
        with open('./functions/tested functions/%s/%s_module.py' % (function_name, function_name), 'w', encoding='utf-8') as f:
            f.write(code)
    
    # 执行该函数
    try:
        exec(code, g)
    except Exception as e:
        print("An error occurred while executing the code:")
        print(e)
    
    # 打印函数名称
    if detail == 0:
        print("The function name is:%s" % function_name)
    
    if detail == 1:
        if tested == False:
            with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), 'r', encoding='utf-8') as f:
                content = f.read()
        else:
            with open('./functions/tested functions/%s/%s_module.py' % (function_name, function_name), 'r', encoding='utf-8') as f:   
                content = f.read()
                
        print(content)
        
    return function_name

而show_functions功能则需要修改为展示文件夹内内文件名称：

In [55]:
def show_functions(tested=False, if_print=False):
    """
    打印tested或untested文件夹内全部函数
    """
    current_directory = os.getcwd()
    if tested == False:
        directory = current_directory + '\\functions\\untested functions'
    else:
        directory = current_directory + '\\functions\\tested functions'
    files_and_directories = os.listdir(directory)
    # 过滤结果，只保留.py文件和非__pycache__文件夹
    files_and_directories = files_and_directories = [name for name in files_and_directories if (os.path.splitext(name)[1] == '.py' or os.path.isdir(os.path.join(directory, name))) and name != "__pycache__"]
    
    if if_print != False:
        for name in files_and_directories:
            print(name)
    
    return files_and_directories

In [5]:
show_functions(tested=True)

['get_email_counts', 'get_latest_email', 'retrieve_emails', 'send_email']

并且，我们可以使用如下方法将每个函数名称同名字符串转化为函数：

In [6]:
globals()['get_email_counts']

<function gptLearning.get_email_counts(userId='me')>

In [7]:
[globals()[name] for name in show_functions(tested=True)]

[<function gptLearning.get_email_counts(userId='me')>,
 <function gptLearning.get_latest_email(userId)>,
 <function gptLearning.retrieve_emails(n, user_id='me')>,
 <function gptLearning.send_email(to, subject, message_text)>]

后续需要将额外添加的remove_to_tested函数，以及修改之后的extract_function_code和show_functions函数写入gptLearning.py文件方便后续进行调用。此外，这里还需要将目前已经定义的四个外部函数全部写入gptLearning.py文件方便后续进行调用。

&emsp;&emsp;至此，我们就完成了经过测试的函数的提示词管理，并搭建了新的提示词管理方法，以及创建了协助完成文件管理的相关函数。接下来我们进一步围绕新函数的创建流程来进行优化。

#### 3.4 code_generate全自动编程函数创建方法

&emsp;&emsp;接下来，我们尝试将此前的从用户需求到LtM提示法创建外部函数完整流程封装到一个名为code_generate的函数内。该函数基本功能、和其他函数交互关系以及和函数库之间调用关系如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/cfba22d96a504f1bb9a234d47878646.png" alt="cfba22d96a504f1bb9a234d47878646" style="zoom:33%;" />

code_generate函数定义过程如下：

In [4]:
def code_generate(req, few_shot='all', model='gpt-4-0613', g=globals(), detail=0):
    """
    Function calling外部函数自动创建函数，可以根据用户的需求，直接将其翻译为Chat模型可以直接调用的外部函数代码。
    :param req: 必要参数，字符串类型，表示输入的用户需求；
    :param few_shot: 可选参数，默认取值为字符串all，用于描述Few-shot提示示例的选取方案，当输入字符串all时，则代表提取当前外部函数库中全部测试过的函数作为Few-shot；\
    而如果输入的是一个包含了多个函数名称的list，则表示使用这些函数作为Few-shot。
    :param model: 可选参数，表示调用的Chat模型，默认选取gpt-4-0613；
    :param g: 可选参数，表示extract_function_code函数作用域，默认为globals()，即在当前操作空间全域内生效；
    :param detail: 可选参数，默认取值为0，还可以取值为1，表示extract_function_code函数打印新创建的外部函数细节；
    :return：新创建的函数名称。需要注意的是，在函数创建时，该函数也会在当前操作空间被定义，后续可以直接调用；
    """
    
    # 提取提示示例的函数名称
    if few_shot == 'all':
        few_shot_functions_name = show_functions(tested=True)
    elif type(few_shot) == list:
        few_shot_functions_name = few_shot
    # few_shot_functions = [globals()[name] for name in few_shot_functions_name]
    
    # 读取各阶段系统提示
    with open('./functions/tested functions/system_messages.json', 'r') as f:
        system_messages = json.load(f)
        
    # 各阶段提示message对象
    few_shot_messages_CM = []
    few_shot_messages_CD = []
    few_shot_messages = []
    
    # 先保存第一条消息，也就是system message
    few_shot_messages_CD += system_messages["system_message_CD"]
    few_shot_messages_CM += system_messages["system_message_CM"]
    few_shot_messages += system_messages["system_message"]

    # 创建不同阶段提示message
    for function_name in few_shot_functions_name:
        with open('./functions/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
            msg = json.load(f)
        few_shot_messages_CD += msg["stage1_CD"]
        few_shot_messages_CM += msg["stage1_CM"]
        few_shot_messages += msg['stage2']
        
    # 读取用户需求，作为第一阶段CD环节User content
    new_req_CD_input = req
    few_shot_messages_CD.append({"role": "user", "content": new_req_CD_input})
    
    print('第一阶段CD环节提示创建完毕，正在进行CD提示...')
    
    # 第一阶段CD环节Chat模型调用过程
    response = openai.ChatCompletion.create(
                  model=model,
                  messages=few_shot_messages_CD
                )
    new_req_pi = response.choices[0].message['content']
    
    print('第一阶段CD环节提示完毕')
    
    # 第一阶段CM环节Messages创建
    new_req_CM_input = new_req_CD_input + new_req_pi
    few_shot_messages_CM.append({"role": "user", "content": new_req_CM_input})
    
    print('第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...')
    # 第一阶段CM环节Chat模型调用过程
    response = openai.ChatCompletion.create(
                      model=model,
                      messages=few_shot_messages_CM
                    )
    new_req_description = response.choices[0].message['content']
    
    print('第一阶段CM环节提示完毕')
    
    # 第二阶段Messages创建过程
    few_shot_messages.append({"role": "user", "content": new_req_description})
    
    print('第二阶段提示创建完毕，正在进行第二阶段提示...')
    
    # 第二阶段Chat模型调用过程
    response = openai.ChatCompletion.create(
                  model=model,
                  messages=few_shot_messages
                )
    new_req_function = response.choices[0].message['content']
    
    print('第二阶段提示完毕，准备运行函数并编写提示示例')
    
    # 提取函数并运行，创建函数名称对象，统一都写入untested文件夹内
    function_name = extract_function_code(s=new_req_function, detail=detail, g=g)
    
    print('新函数保存在./functions/untested functions/%s/%s_module.py文件中' % (function_name, function_name))
    
    # 创建该函数提示示例
    new_req_messages_CD = [
                          {"role": "user", "content": new_req_CD_input},
                          {"role": "assistant", "content": new_req_pi}
                         ]
    new_req_messages_CM = [
                          {"role": "user", "content": new_req_CM_input},
                          {"role": "assistant", "content":new_req_description}
                         ]
    
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), encoding='utf-8') as f:
        new_req_function = f.read()
    
    new_req_messages = [
                       {"role": "user", "content": new_req_description},
                       {"role": "assistant", "content":new_req_function}
                      ] 
    
    new_req_prompt = {
                     "stage1_CD": new_req_messages_CD,
                     "stage1_CM": new_req_messages_CM,
                     "stage2": new_req_messages
                    }   
    
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'w') as f:
        json.dump(new_req_prompt, f)
        
    print('新函数提示示例保存在./functions/untested functions/%s/%s_prompt.json文件中' % (function_name, function_name))
    print('done')
    return function_name

---

&emsp;&emsp;这里我们可以尝试手动验证上述流程：

In [42]:
function_name = 'get_latest_email'

In [43]:
with open('./functions/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
    msg = json.load(f)

In [44]:
msg['stage2']

[{'role': 'user',
  'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'assistant',
  'content': 'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的一封最新邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'messages\', [])\n\n    # 遍历邮件\n    for message in messages:\n        # 获取邮件的详细信息\n        msg = service.users().me

In [38]:
for function_name in few_shot_functions_name:
    with open('./functions/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
        msg = json.load(f)
    few_shot_messages_CD += msg["stage1_CD"]
    few_shot_messages_CM += msg["stage1_CM"]
    few_shot_messages += msg['stage2']

In [39]:
few_shot_messages_CD

[{'role': 'system',
  'content': '为了更好编写满足用户需求的python函数，我们需要先识别用户需求中的变量，以作为python函数的参数。需要注意的是，当前编写的函数中涉及到的邮件收发查阅等功能，都是通过调用Gmail API来完成。'},
 {'role': 'user', 'content': '请帮我查下Gmail邮箱里最后一封邮件内容。'},
 {'role': 'assistant', 'content': '当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
 {'role': 'user',
  'content': '请帮我给陈明发送一封Gmail邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。'},
 {'role': 'assistant',
  'content': '当前需求中可以作为函数参数的是：1.发送邮件的对象；2.发送邮件的主题；3.邮件具体内容'}]

In [40]:
few_shot_messages_CM

[{'role': 'system',
  'content': '我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。'},
 {'role': 'user',
  'content': '请帮我查下Gmail邮箱里最后一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
 {'role': 'assistant',
  'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'user',
  'content': '请帮我给陈明发送一封Gmail邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。当前需求中可以作为函数参数的是：1.发送邮件的对象；2.发送邮件的主题；3.邮件具体内容'},
 {'role': 'assistant',
  'content': '请帮我编写一个python函数，用于给陈明发送邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划，函数要求如下：                  1.函数参数to、subject和message_text，三个参数都是字符串类型，其中to表示发送邮件对象，subject表示邮件主题，message_text表示邮件具体内容；                  2.函数返回结果是当前邮件发送状态，返回结果本身必须是一个json格式对象；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用

In [41]:
few_shot_messages

[{'role': 'system',
  'content': '我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。'},
 {'role': 'user',
  'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'assistant',
  'content': 'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的一封最新邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'m

In [16]:
new_req_CD_input = "请查下我的邮箱里是否有来自端木天的未读邮件，有的话请解读下这封未读邮件的内容。"

In [43]:
few_shot_messages_CD.append({"role": "user", "content": new_req_CD_input})
few_shot_messages_CD

[{'role': 'system',
  'content': '为了更好编写满足用户需求的python函数，我们需要先识别用户需求中的变量，以作为python函数的参数。需要注意的是，当前编写的函数中涉及到的邮件收发查阅等功能，都是通过调用Gmail API来完成。'},
 {'role': 'user', 'content': '请帮我查下Gmail邮箱里最后一封邮件内容。'},
 {'role': 'assistant', 'content': '当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
 {'role': 'user',
  'content': '请帮我给陈明发送一封Gmail邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。'},
 {'role': 'assistant',
  'content': '当前需求中可以作为函数参数的是：1.发送邮件的对象；2.发送邮件的主题；3.邮件具体内容'},
 {'role': 'user', 'content': '请查下我的邮箱里是否有来自端木天的未读邮件，有的话请解读下这封未读邮件的内容。'},
 {'role': 'user', 'content': '请查下我的邮箱里是否有来自端木天的未读邮件，有的话请解读下这封未读邮件的内容。'}]

第一阶段CD环节提示：

In [39]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=few_shot_messages_CD
)

In [41]:
new_req_pi = response.choices[0].message['content']
new_req_pi

'当前需求中可以作为函数参数的是：1.收件人邮箱；2.邮件的发送者；3.邮件的状态。'

第一阶段CM环节提示：

In [44]:
new_req_CM_input = new_req_CD_input + new_req_pi
new_req_CM_input

'请查下我的邮箱里是否有来自端木天的未读邮件，有的话请解读下这封未读邮件的内容。当前需求中可以作为函数参数的是：1.收件人邮箱；2.邮件的发送者；3.邮件的状态。'

In [46]:
few_shot_messages_CM.append({"role": "user", "content": new_req_CM_input})
few_shot_messages_CM

[{'role': 'system',
  'content': '我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。'},
 {'role': 'user',
  'content': '请帮我查下Gmail邮箱里最后一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
 {'role': 'assistant',
  'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'user',
  'content': '请帮我给陈明发送一封Gmail邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划。当前需求中可以作为函数参数的是：1.发送邮件的对象；2.发送邮件的主题；3.邮件具体内容'},
 {'role': 'assistant',
  'content': '请帮我编写一个python函数，用于给陈明发送邮件，请他明天早上9点半来我办公室开会，商量下半年技术开发计划，函数要求如下：                  1.函数参数to、subject和message_text，三个参数都是字符串类型，其中to表示发送邮件对象，subject表示邮件主题，message_text表示邮件具体内容；                  2.函数返回结果是当前邮件发送状态，返回结果本身必须是一个json格式对象；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用

In [47]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=few_shot_messages_CM
)

In [48]:
new_req_description = response.choices[0].message['content']
new_req_description

"请帮我编写一个python函数，用于查询我的邮箱中是否有端木天的未读邮件，如有，请解读这封邮件的内容，函数要求如下：                  1.函数参数userId, sender, labelIds，其中userId表示邮件接收者（我的邮箱），sender表示邮件发送者，labelIds表示邮件状态，它是一个列表，列表中元素为'UNREAD'表示未读邮件；                  2.函数返回结果是一个包含邮件信息的json格式对象，如果没有找到邮件，返回结果中包含相关提示信息；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

第二阶段提示

In [50]:
few_shot_messages.append({"role": "user", "content": new_req_description})
few_shot_messages

[{'role': 'system',
  'content': '我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。'},
 {'role': 'user',
  'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
 {'role': 'assistant',
  'content': 'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的一封最新邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'m

In [51]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=few_shot_messages
)

In [52]:
new_req_function = response.choices[0].message['content']
new_req_function

'def get_unread_email(userId, sender, labelIds):\n    """\n    查询指定发件人的未读邮件\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :param sender: 必要参数，字符串类型，用于表示需要查询的发件人，输入发件人的邮箱地址；\n    :param labelIds: 必要参数，列表类型，用于表示需要查询的邮件状态，输入[\'UNREAD\']查询未读邮件；\n    :return：返回查询到的邮件内容，如果没有找到相关邮件，则返回提示信息；\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n\n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n\n    # 列出用户的未读邮件\n    results = service.users().messages().list(userId=userId, labelIds=labelIds, q=f\'from:{sender}\').execute()\n    messages = results.get(\'messages\', [])\n\n    if not messages:\n        print(\'No messages found.\')\n        return json.dumps({\'message\':\'No messages found.\'})\n\n    # 遍历邮件\n    for message in messages:\n        # 获取邮件的详细信息\n        msg = service.users().messages().get(userId=userId, id=message[\'id\']).execute()\n        \n    return json.dumps

In [13]:
extract_function_code(new_req_function, detail=1, g=globals())

def get_unread_email(userId, sender, labelIds):
    """
    查询指定发件人的未读邮件
    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\
    注意，当查询我的邮箱时，userId需要输入'me'；
    :param sender: 必要参数，字符串类型，用于表示需要查询的发件人，输入发件人的邮箱地址；
    :param labelIds: 必要参数，列表类型，用于表示需要查询的邮件状态，输入['UNREAD']查询未读邮件；
    :return：返回查询到的邮件内容，如果没有找到相关邮件，则返回提示信息；
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')

    # 创建 Gmail API 客户端
    service = build('gmail', 'v1', credentials=creds)

    # 列出用户的未读邮件
    results = service.users().messages().list(userId=userId, labelIds=labelIds, q=f'from:{sender}').execute()
    messages = results.get('messages', [])

    if not messages:
        print('No messages found.')
        return json.dumps({'message':'No messages found.'})

    # 遍历邮件
    for message in messages:
        # 获取邮件的详细信息
        msg = service.users().messages().get(userId=userId, id=message['id']).execute()
        
    return json.dumps(msg)



---

接下来测试code_generate函数效果。我们还是以“查询和端木天之间的未读邮件”为需求，以'get_latest_email'和'send_email'两个函数为提示示例，借助code_generate一步到位创建外部函数：

In [150]:
few_shot_functions = ['get_latest_email', 'send_email']

In [151]:
req = "请查下我的邮箱里是否有来自端木天的未读邮件，有的话请解读下这封未读邮件的内容。"

In [152]:
function_name = code_generate(req=req, few_shot=few_shot_functions)

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:check_unread_mail
新函数保存在./functions/untested functions/check_unread_mail/check_unread_mail_module.py文件中
新函数提示示例保存在./functions/untested functions/check_unread_mail/check_unread_mail_prompt.json文件中
done


然后我们手动对新创建的check_unread_mail函数进行功能验证：

In [153]:
functions_list = [check_unread_mail]

In [154]:
messages = [{"role": "system", "content": "端木天的邮箱地址是:2323365771@qq.com"},
            {"role": "user", "content": req}]

In [155]:
final_response = run_conversation(messages=messages, functions_list=functions_list, model="gpt-4-0613")

In [156]:
final_response

'您有一封来自端木天的未读邮件，主题是“本周工作进度”，内容如下：“下半年技术开发计划已做好，随时可以向您进行汇报。”。'

能够发现，上述流程能够非常高效便捷的进行外部函数的创建。

#### 3.5 基于code_generate编写更多外部功能函数

&emsp;&emsp;为了测试code_generate是否能够更好的满足更多的外部函数编写需求，我们这里再进行一些额外新增需求测试，尝试让code_generate自动帮我们编写检查第一封邮件信息内容的函数，以及整理和端木天之间的邮件记录。具体测试流程如下。

- 检测第一封邮件收件时间和邮件内容

&emsp;&emsp;首先我们让code_generate帮我们创建一个外部函数来查阅第一封邮件收件时间和邮件内容，不难发现这其实是一个相对简单的功能需求，这里我们仍然以get_latest_email和send_email作为提示示例，具体测试过程如下：

In [5]:
few_shot_functions = ['get_latest_email', 'send_email']

In [18]:
req = "请查下我的邮箱里第一封邮件，并告诉我第一封邮件的收件时间和邮件内容"

In [19]:
function_name = code_generate(req=req, few_shot=few_shot_functions)

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:get_first_email_info
新函数保存在./functions/untested functions/get_first_email_info/get_first_email_info_module.py文件中
新函数提示示例保存在./functions/untested functions/get_first_email_info/get_first_email_info_prompt.json文件中
done


此时创建的函数名称为get_first_email，并且在untested文件夹内保存了相关代码和提示内容。接下来我们直接带入测试该函数功能能否满足要求：

In [20]:
functions_list = [get_first_email_info]

In [21]:
messages = [{"role": "user", "content": req}]

In [22]:
messages

[{'role': 'user', 'content': '请查下我的邮箱里第一封邮件，并告诉我第一封邮件的收件时间和邮件内容'}]

In [23]:
final_response = run_conversation(messages=messages, functions_list=functions_list, model="gpt-4-0613")

In [24]:
final_response

'您的第一封邮件在2023年7月23日12:15接收。邮件的内容是关于您的辅助邮箱ksken166@gmail.com与2323365771@qq.com已通过验证的信息。邮件还提供了一个链接（https://myaccount.google.com/notifications）以查看安全性活动，并解释说这封电子邮件是为了让您了解有关您的Google帐户和服务的重大变化。'

能够发现，当前get_first_email函数能够很好的满足该需求。

> 需要注意的是，在围绕当前需求进行函数创建时首次创建的函数并不能正常运行，错误原因在于使用了一个未经定义的变量，错误代码如下：<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/0d996fecb50db908512733f81f00eb8.png" alt="0d996fecb50db908512733f81f00eb8" style="zoom:33%;" />

> 测试环节的所有报错函数都将统一进行归纳和整理，这里我们独将code_generate创建函数时的报错函数统一保存在functions/wrong code文件夹内，下一小节将重点探讨如何进行设置全自动debug流程来大幅提高大语言模型编程稳定性。

- 整理和端木天之间的邮件记录

&emsp;&emsp;接下来进行第二项测试，即尝试让code_generate函数自动编写函数，来整理我和端木天之间的邮件通信记录，并对其内容进行总结。很明显，该需求难度略高于第一个测试问题。考虑到这个需求难度更大，我们尝试带入全部经过测试的函数作为提示示例，即设置few_shot='all'。具体的外部函数自动创建与验证过程如下：

In [71]:
req = "帮我整理下我和端木天之间的邮件通信记录"

In [63]:
function_name = code_generate(req=req, few_shot='all')

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:get_emails_between
新函数保存在./functions/untested functions/get_emails_between/get_emails_between_module.py文件中
新函数提示示例保存在./functions/untested functions/get_emails_between/get_emails_between_prompt.json文件中
done


此时创建的函数名称为get_emails_between，代码和提示文本都保存在untested文件夹内。接下来我们直接带入该函数进行功能测试

In [64]:
functions_list = [get_emails_between]

In [68]:
messages = [{"role": "system", "content": "端木天的邮箱地址是:2323365771@qq.com"},
            {"role": "system", "content": "我的邮箱地址是:ksken166@gmail.com"},
            {"role": "user", "content": req}]

In [69]:
final_response = run_conversation(messages=messages, functions_list=functions_list, model="gpt-4-0613")

In [72]:
final_response

'你和端木天之间有9封的邮件通信记录，具体日期如下：\n1. 2023年7月23日 6:13:48\n2. 2023年7月21日 7:38:55\n3. 2023年7月21日 7:27:55\n4. 2023年7月21日 7:24:13\n5. 2023年7月21日 7:21:39\n6. 2023年7月21日 7:14:38\n7. 2023年7月21日 7:14:22\n8. 2023年7月21日 6:51:26\n9. 2023年7月21日 6:08:02'

能够发现get_emails_between函数能够很好的汇总我和端木天之间的通信记录。

> 需要注意的是，在反复多次创建该函数的过程中，code_generate也创建了一个错误的函数，函数内容如下，很明显模型在创建该函数时并未很好的理解“编写函数说明文档”是什么意思，导致函数本身没有说明文档，最终导致无法自动翻译为外部函数，最终导致运行报错：<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/7ae14ff8bd5bdf57d72e90855d5c503.png" alt="7ae14ff8bd5bdf57d72e90855d5c503" style="zoom:33%;" />

&emsp;&emsp;至此，我们就完整测试了code_generate函数功能。总的来看，基于LtM提示流程的gpt-4自动编程函数还是能够自动创建符合要求的函数，唯一的问题就是gpt-4的编程能力发挥的不是很稳定。尽管gpt-4模型是目前最强的编程模型之一，但在实际进行批量代码创建时，仍然有可能出现各类问题。除了上面所列举的诸如模型无法很好的理解“编写注释”或者直接使用了未经定义的变量之外，还有可能出现很多类似于代码结构错误导致无法被extract函数提取源码、使用了过期的函数库等各类问题，在使用code_generate函数进行自动编程时，我们仍然需要实时手动检验，才能确保最终的函数是可用的。但无论如何，从编程流程上来看，拥有诸多辅助函数加持下的code_generate函数已经能够以一个非常高的效率进行外部函数创建，相比于手动编写函数来说，哪怕是需要人工debug，当前的基于code_generate函数的函数编写流程也要高效很多。至此，我们就完成了一个相对较为稳定的全自动函数编写流程的创建。

- 大语言模型编程能力不稳定问题及解决方法

&emsp;&emsp;需要注意的是，涌现能力的不稳定性是大语言模型在使用过程中最大的问题，对于大语言模型来说，不仅是编程能力“不太稳定”，很多时候围绕复杂问题的“推理能力”、甚至是围绕对话的“问答能力”都不稳定。因此，如何才能让大语言模型稳定的解决某些问题，就成了大模型应用过程的最核心的问题之一。其实从本质上来说，我们之所以借助一些提示工程的手段来进行自动编程，也是为了提升大语言模型的编程稳定性，而后续我们要介绍的微调方法，本质上也是希望能够更好的激发模型的涌现能力，让大语言模型“稳定的”解决某些问题。

&emsp;&emsp;当然，大语言模型本身能否稳定的输出正确结果，其实也是和当前问题难度有较大关系，大语言模型对于简单问题往往具备较强的解决能力，但对于复杂问题，则会表现出不太稳定、甚至是完全无法解决问题的情况，此时我们就需要围绕这一问题考虑采用一系列的解决方法。而除了提示工程和微调之外，其实还有一些更加偏工业应用的解决方案。这里所谓的偏工业应用的方法，指的是工程师们在长期实际使用大模型的过程中总结经验得出的一些方法，这些方法可能并不具备非常深厚的理论基础，但切实有效（就类似于机器学习领域的模型融合）。这些方法中最简单的方法就是让模型反复多次运行，有的时候我们不需要进行任何额外的调整，只需要反复多次运行，就可以让模型顺利输出一个正确的编程结果。上述测试环节中第一个函数——最终可以顺利运行的get_first_email函数其实就是通过反复多次运行code_generate函数得到的结果。此外，添加更多的、差异性较大的提示示例也是非常有效的方法，例如上述测试环节中第二个函数get_emails_between就是在添加了更多的提示示例，并且反复运行两次之后得到的函数。

&emsp;&emsp;此外，还有一种方法也至关重要，该方法也能大幅提高大语言模型编程稳定性——那就是创建基于大语言模型的自动debug流程，让大语言模型自纠自查，来提高编程稳定性。需要注意的是，在工业应用过程中，大语言模型的自纠自查是非常重要的提高模型稳定性的方法，能够有效提升模型围绕标准答案进行回答的能力。而在下一小节中，我们就将详细介绍如何借助大语言模型构建代码自动debug流程，并且同时，在code_generate基础之上，搭建一个更加自动化的编程流程。

---

**<center>【总结】提升大语言模型编程稳定性方法**

&emsp;&emsp;抛开工业方法还是理论方法不谈，总的来说，提升大语言模型编程稳定性的方法可以按照如下方式进行总结：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/%E5%A6%82%E4%BD%95%E6%9B%B4%E5%A5%BD%E7%9A%84%E5%BC%95%E5%AF%BC%E5%A4%A7%E8%AF%AD%E8%A8%80%E6%A8%A1%E5%9E%8B%E8%BF%9B%E8%A1%8C%E7%BC%96%E7%A8%8B.png" alt="如何更好的引导大语言模型进行编程" style="zoom:33%;" />

当然，其中很多方法目前还没讲，待下一小节讲完更多方法之后，我们再进行系统性总结。

---